In [1]:
from typing import List
import warnings

from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

### Define signature of a generic activation function class.

In [2]:
from abc import ABC, abstractmethod
class ActivationFunction(ABC):
    def __init__(self):
        pass
    @abstractmethod
    def function(self):
        pass
    @abstractmethod
    def vectorized_function(self):
        pass
    @abstractmethod
    def derivative(self):
        pass
    @abstractmethod
    def __call__(self):
        pass

### Define all activation functions

In [62]:
class Sigmoid(ActivationFunction):
    def __call__(self, x):
        return self.vectorized_function(x)
    def function(self, x):
        x = np.clip(x, -700, 700)
        return 1/(1+np.exp(-x))
    def vectorized_function(self, x):
        return np.vectorize(self.function)(x)
    def derivative(self, x):
        return self.vectorized_function(x)*(1-self.vectorized_function(x))
    def vectorized_derivative(self, x):
        return self.vectorized_function(x)*(1-self.vectorized_function(x))

class Tanh(ActivationFunction):
    def __call__(self, x):
        return self.vectorized_function(x)
    def function(self, x):
        x = np.clip(x, -100, 100)
        # print(x)
        expr1, expr2 = np.exp(x), np.exp(-x)
        return (expr1-expr2)/(expr1+expr2)
        
    def vectorized_function(self, x):
        return np.vectorize(self.function)(x)
    def derivative(self, x):
        expr1, expr2 = np.exp(x), np.exp(-x)
        return 4/((expr1+expr2)**2)
    def vectorized_derivative(self, x):
        return self.derivative(x)

class ReLU(ActivationFunction):
    def __call__(self, x):
        return self.vectorized_function(x)
    def function(self, x):
        return max(0, x)
    def derivative(self, x):
        if max(0,x) == 0:
            return 0
        return 1
    def vectorized_function(self, x):
        return np.vectorize(self.function)(x)
    def vectorized_derivative(self, x):
        return np.vectorize(self.derivative)(x)

class LeakyReLU(ActivationFunction):
    def __call__(self, x):
        return self.vectorized_function(x)
    def function(self, x):
        return max(0.01*x, x)
    def derivative(self, x):
        if max(0.01*x, x) == x:
            return 1
        return 0.01
    def vectorized_function(self, x):
        return np.vectorize(self.function)(x)
    def vectorized_derivative(self, x):
        return np.vectorize(self.derivative)(x)

class Linear:
    def __call__(self, x):
        return x
    def vectorized_derivative(self, x):
        return np.ones_like(x)

### Import existing loss functions and code new ones

In [4]:
def huberLoss(y_true, y_pred, delta=10):
    err = y_true - y_pred
    abs_err = np.abs(err)
    delta_sq = 0.5*(delta ** 2)
    huber_loss_vectorized = np.vectorize(lambda x: (x**2)*0.5 if x <= delta else delta*x - delta)
    huber_loss_vec = huber_loss_vectorized(abs_err)
    return np.sum(huber_loss_vec)
    # return np.sum(huber_loss_vectorized(abs_err))

In [93]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, log_loss

class Crossentropy:
    def __call__(self, y_true, y_pred):
        return self.function(y_true, y_pred)
    def function(self, y_true, y_pred):
        return log_loss(y_true, y_pred, labels = np.arange(y_pred.shape[1]))
    def derivative(self, y_true, y_pred):
        # return an n x 1 matrix
        epsilon = 1e-6
        y_pred_capped = np.clip(y_pred, epsilon, 1-epsilon) # cap predicted probabilities to avoid floating point issues after taking reciprocal
        y_pred_inv = 1/y_pred_capped # 1/y_hat
        n_classes = y_pred.shape[1]
        y_true_proba = np.eye(n_classes)[y_true] # for ease of computing the derivative, basically a one-hot encoding
        derivative_loss_arr = -np.multiply(y_true_proba, y_pred_inv)# -[y/y_hat, (1-y)/(1-y_hat)]
        derivative_loss_arr = np.sum(derivative_loss_arr, axis=1) # sum up, i.e., y/y_hat + (1-y)/(1-y_hat)  
        return derivative_loss_arr

class MSE:
    def __call__(self, y_true, y_pred):
        return self.function(y_true, y_pred)
    def function(self, y_true, y_pred):
        return mean_squared_error(y_true, y_pred)
    def derivative(self, y_true, y_pred):
        y_true_reshaped = y_true.copy()
        y_true_reshaped = y_true_reshaped.reshape(-1, 1)
        return -2*(y_true_reshaped - y_pred)

class MAE:
    def __call__(self, y_true, y_pred):
        return self.function(y_true, y_pred)
    def function(self, y_true, y_pred):
        return mean_absolute_error(y_true, y_pred)
    def derivative(self, y_true, y_pred):
        y_true_reshaped = y_true.copy()
        y_true_reshaped = y_true_reshaped.reshape(-1, 1)
        return np.where(y_true > y_pred, 1, -1)

class HuberLoss:
    def __init__(self, delta=10):
        self.delta=delta
    def __call__(self, y_true, y_pred):
        return self.function(y_true, y_pred)
    def function(self, y_true, y_pred):
        return huberLoss(y_true, y_pred, self.delta)
    def derivative(self, y_true, y_pred):
        y_true_reshaped = y_true.copy()
        y_true_reshaped = y_true_reshaped.reshape(-1, 1)
        err = y_true_reshaped - y_pred
        huber_loss_derivative_vectorized = np.vectorize(lambda x: x if np.abs(x) <= self.delta else -self.delta if x < 0 else self.delta)
        return huber_loss_derivative_vectorized(err)

#### Test on binary classification

In [ ]:
y_true, y_pred = np.random.randint(0, 2, size=(3,)), np.random.uniform(0,1,(3,1))
print(y_true,"\n\n", y_pred)
y_pred_proba = np.hstack([y_pred, 1-y_pred])
print(y_pred_proba)

In [ ]:
ce = Crossentropy()
print(f"Loss = {round(ce(y_true, y_pred_proba), 2)}, Derivative = {ce.derivative(y_true, y_pred_proba)}.")

#### Test on Multi-class classification

In [ ]:
y_true_multiclass, y_pred_multiclass_preprocessed = np.random.randint(0, 3, size=(5,)), np.random.uniform(0,1,(5, 3))
y_pred_multiclass = y_pred_multiclass_preprocessed / y_pred_multiclass_preprocessed.sum(axis=1, keepdims=True)
print(f"{y_true_multiclass}\n\n{y_pred_multiclass}\n")

In [ ]:
print(f"Loss = {round(ce(y_true_multiclass, y_pred_multiclass), 2)}, Derivative = {ce.derivative(y_true_multiclass, y_pred_multiclass), 2}.")

Rest assured, the return value is a numpy.ndarray

#### Test on Regression using Mean Squared Error

In [ ]:
yt, yp = np.random.randint(0,10,(4,)), np.random.randint(0,10,(4,))
print(f"{yt}\n\n{yp}")

In [ ]:
mse_loss_fn = MSE()
print(f"Loss = {mse_loss_fn(yt, yp)}\nderivative = \n{mse_loss_fn.derivative(yt, yp)}\n")

#### Test on Regression using Mean Absolute Error

In [ ]:
mae_loss_fn = MAE()
print(f"Loss = {mae_loss_fn(yt, yp)}\nderivative = \n{mae_loss_fn.derivative(yt, yp)}\n")

#### Test on Regression using Huber Loss

In [ ]:
huber_loss_fn = HuberLoss(delta=8)
print(f"Delta = {huber_loss_fn.delta}, Loss = {huber_loss_fn(yt, yp)}\nderivative = \n{huber_loss_fn.derivative(yt, yp)}\n")

### Define Layer and Sequential Model

In [94]:
class Layer():
    
    __valid_activations = {'sigmoid': Sigmoid, 'tanh': Tanh, 'relu': ReLU, 'leaky_relu': LeakyReLU, 'linear': Linear}
    
    def __init__(self, activation, in_dim, out_dim, learning_rate=0.01):
        if activation.lower() not in list(Layer.__valid_activations.keys()):
            raise Exception(f"Valid activations are {Layer.__valid_activations}.")
        self.activation = Layer.__valid_activations[activation.lower()]()
        self.learning_rate = learning_rate
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.weights = np.random.uniform(-1,1, size=(in_dim, out_dim))

    def forward_compute(self, X, compute_gradient=False):
        output_prime = X.dot(self.weights) # of order n x out_dim
        output_val = self.activation(output_prime) # of order n x out_dim

        if compute_gradient:
            # computing stuff for eventual backpropagation
            self._activation_gradient = self.activation.vectorized_derivative(output_prime) # of order n x out_dim
            self._input = X
        
        return output_val

    def backprop_compute(self, prev_grad_multipliers, verbose=False):
        gradient_mat = 1
        
        # To Do: find a way of multiplying pre v_grad_multipliers with this layer's gradient multiplier matrix.
        '''
        Needs: 
         1. current-layer's activation gradient matrix(as a function of this layer's input, i.e. prev layers output)
         2. current-layer's input matrix
        '''

        activ_prev_layer_output_element_wise_product = np.multiply(prev_grad_multipliers, self._activation_gradient) # of order n x out_dim
        weights_gradient = self._input.T.dot(activ_prev_layer_output_element_wise_product)
        self.weights -= self.learning_rate * weights_gradient
        if verbose:
            print(f"\tElementwise product of next layer output and this layer's activation gradient = \n{activ_prev_layer_output_element_wise_product}\n\n")
            print(f"\tWeight Gradient = \n{weights_gradient}\n\n")

        send_mat_to_prev_layer = activ_prev_layer_output_element_wise_product.dot(self.weights.T) # of order n x in_dim, 
                                                                      # which is basically n x out_dim for the previous layer.
        return send_mat_to_prev_layer

In [95]:
class ModelSequential():
    __valid_loss_functions = {
        'crossentropy': Crossentropy, 
        'mse': MSE,
        'mae': MAE,
        'huber': HuberLoss
    }

    # def __init__(self, layers_arr: List, metrics_to_track, early_stopping=False):
    def __init__(self, layers_arr: List[Layer]):
        self._layers = layers_arr

    def compile(self, loss_function, n_iter = 100):
        if loss_function.lower() not in ModelSequential.__valid_loss_functions:
            raise Exception(f"Loss functions should be any of {ModelSequential.__valid_loss_functions.keys()}.")
        
        self.loss_function = ModelSequential.__valid_loss_functions[loss_function.lower()]()
        self.n_iter = n_iter

    def reweight_layers(self):
        n_layers = len(self._layers)
        for i in range(n_layers):
            in_dim, out_dim = self._layers[i].in_dim, self._layers[i].out_dim
            self._layers[i].weights = np.random.uniform(-1,1, size=(in_dim, out_dim))
    def fit(self, X, y, verbose=False):
        pbar_iterations = tqdm(self.n_iter)
        pbar_iterations.set_description("#Iterations: ")
        for iter_ in range(self.n_iter):
            n_layers = len(self._layers)
            input_for_next_layer = X.copy()


            # forward compute
            for i in range(n_layers):
                try:
                    input_for_next_layer = self._layers[i].forward_compute(input_for_next_layer, compute_gradient=True)
                except ValueError as ve:
                    raise Exception(f"Forward compute failed at layer {i} with the following exception:\n{ve}")
                except RuntimeWarning as rw:
                    raise Exception(f"Forward compute failed at layer {i} with the following exception:\n {rw}")

            y_pred = self.predict(X)
            input_for_next_layer = self.loss_function.derivative(y_true=y, y_pred=y_pred)

            if verbose:
                print(f"Gradient of loss function has shape {input_for_next_layer.shape} and is = \n{input_for_next_layer}\n\n")

        
            # backprop
            for i in range(n_layers-1, -1, -1):
                if verbose:
                    print(f"Handling layer {i}...")
                with warnings.catch_warnings():
                    warnings.filterwarnings("error")
                    try:
                        input_for_next_layer = self._layers[i].backprop_compute(input_for_next_layer, verbose)
                    except ValueError as ve:
                        return Exception(f"Backpropagation failed at layer {i} with the following exception:\n{ve}")
                    except RuntimeWarning:
                        raise Exception(f"Forward compute failed at layer {i} with the following exception:\n {ve}")

            pbar_iterations.update(1)
        pbar_iterations.close()
    
    def predict(self, X):
        n_layers = len(self._layers)
        input_for_next_layer = X.copy()

        # forward compute
        for i in range(n_layers):
            with warnings.catch_warnings():
                try:
                    input_for_next_layer = self._layers[i].forward_compute(input_for_next_layer, compute_gradient=True)
                except ValueError as ve:
                    raise Exception(f"Forward compute failed at layer {i} with the following exception:\n{ve}")
                except RuntimeWarning as rw:
                    raise Exception(f"Forward compute failed at layer {i} with the following exception:\n {rw}")
        return input_for_next_layer

### Test

In [8]:
import sys

def get_y(x):
    return x[:,0] + (x[:,1]**2) + np.abs(x[:, 2])

x_10K = np.random.randint(-100,100,size=(10000, 3))
y_10K = get_y(x_10K)

# Column-wise normalization using L2 normalization (Euclidean norm)
x_10K_normalized = x_10K / np.linalg.norm(x_10K, axis=0)  # Normalizing along axis 0 (columns)

x_100K = np.random.randint(-100,100,size=(100000, 3))
y_100K = get_y(x_10K)

x_1M = np.random.randint(-100,100,size=(1000000, 3))
y_1M = get_y(x_1M)

x_10M = np.random.randint(-100,100,size=(10000000, 3))
y_10M = get_y(x_10M)

print(sys.getsizeof(x_10K), sys.getsizeof(x_100K), sys.getsizeof(x_1M), sys.getsizeof(x_10M))
print()
print(sys.getsizeof(y_10K), sys.getsizeof(y_100K), sys.getsizeof(y_1M), sys.getsizeof(x_10M))

240120 2400120 24000120 240000120

80104 80104 8000104 240000120


In [96]:
l1, l2, l3 = Layer('sigmoid', x_10K.shape[1], 3), Layer('leaky_relu', 3, 2), Layer('linear', 2, 1)

In [97]:
model = ModelSequential([l1, l2, l3])

In [98]:
# model.compile(loss_function='huber', n_iter=10)
model.compile(loss_function='mse', n_iter=10)

In [30]:
init_pred_10k = model.predict(x_10K_normalized)
print(model.loss_function(y_true=y_10K, y_pred=init_pred_10k))

3321433549345.393


In [99]:
n_layers = len(model._layers)
for i in range(n_layers):
    print(f"Layer {i}\n{model._layers[i].weights}\n\n")

Layer 0
[[-0.95935186  0.04597412  0.20524535]
 [-0.55453881 -0.72329169  0.2728556 ]
 [ 0.86915297  0.06818314 -0.97932693]]


Layer 1
[[-0.80107311 -0.6941181 ]
 [ 0.86440511 -0.91224689]
 [ 0.77003371  0.7647649 ]]


Layer 2
[[0.8650483 ]
 [0.79471011]]




In [100]:
model.reweight_layers()
model.fit(x_10K_normalized, y_10K, verbose=True)

0it [00:00, ?it/s]

Gradient of loss function has shape (10000, 1) and is = 
[[ -320.00871465]
 [-1746.00870167]
 [-4800.00872172]
 ...
 [-7846.00869045]
 [-2682.0087321 ]
 [-1988.00870162]]


Handling layer 2...
	Elementwise product of next layer output and this layer's activation gradient = 
[[ -320.00871465]
 [-1746.00870167]
 [-4800.00872172]
 ...
 [-7846.00869045]
 [-2682.0087321 ]
 [-1988.00870162]]


	Weight Gradient = 
[[358560.88925969]
 [348545.51446022]]


Handling layer 1...
	Elementwise product of next layer output and this layer's activation gradient = 
[[ 11472.942497    11152.46298015]
 [ 62597.84973414  60849.27227515]
 [172089.76358354 167282.6930084 ]
 ...
 [281294.85984139 273437.30797123]
 [ 96155.29374922  93469.33920085]
 [ 71274.0262148   69283.0927214 ]]


	Weight Gradient = 
[[1.19072453e+09 1.15746342e+09]
 [1.19069722e+09 1.15743687e+09]
 [1.19112935e+09 1.15785693e+09]]


Handling layer 0...
	Elementwise product of next layer output and this layer's activation gradient = 
[[-6

UnboundLocalError: local variable 've' referenced before assignment

In [70]:
y_pred_10k = model.predict(x_10K_normalized)
model.loss_function(y_true=y_10K, y_pred=y_pred_10k)

4.901231073506451e+123

### Print all layer's weights

In [101]:
n_layers = len(model._layers)
for i in range(n_layers):
    print(f"Layer {i}\n{model._layers[i].weights}\n\n")

Layer 0
[[-4.09700621e+09 -4.09391876e+09 -4.09675213e+09]
 [-5.42963388e+10 -5.42931961e+10 -5.43142585e+10]
 [-2.09461301e+10 -2.09455371e+10 -2.09523633e+10]]


Layer 1
[[-inf -inf]
 [-inf -inf]
 [-inf -inf]]


Layer 2
[[1.39158237e+290]
 [1.35271060e+290]]




In [40]:
print(dir(model))

['_ModelSequential__layers', '_ModelSequential__valid_loss_functions', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'compile', 'fit', 'loss_function', 'n_iter', 'predict']


In [ ]:
import sys
print(sys.getsizeof(model))